In [20]:
import sys
import pandas as pd
import json
sys.path.append("..")
from utils.data_preprocessing import preprocess_text
from utils.feature_extraction import bag_of_words, tfidf_features, extract_embeddings

In [21]:

train_path = '../dataset/PIZZA_train.json'
test_path = '../dataset/PIZZA_dev.json'
def read_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            try:
                record = json.loads(line.strip())
                data.append(record)
                
                # Process in chunks of 10,000 records
                if i > 0 and i % 10000 == 0:
                    print(f"Processed {i} records so far...")
            except json.JSONDecodeError:
                continue
    return data

# Convert remaining data to DataFrame
data = read_data(train_path)
if data:
    df = pd.DataFrame(data)
    
data = read_data(test_path)
if data:
    dev = pd.DataFrame(data)


Processed 10000 records so far...
Processed 20000 records so far...
Processed 30000 records so far...
Processed 40000 records so far...
Processed 50000 records so far...
Processed 60000 records so far...
Processed 70000 records so far...
Processed 80000 records so far...
Processed 90000 records so far...
Processed 100000 records so far...
Processed 110000 records so far...
Processed 120000 records so far...
Processed 130000 records so far...
Processed 140000 records so far...
Processed 150000 records so far...
Processed 160000 records so far...
Processed 170000 records so far...
Processed 180000 records so far...
Processed 190000 records so far...
Processed 200000 records so far...
Processed 210000 records so far...
Processed 220000 records so far...
Processed 230000 records so far...
Processed 240000 records so far...
Processed 250000 records so far...
Processed 260000 records so far...
Processed 270000 records so far...
Processed 280000 records so far...
Processed 290000 records so f

In [22]:
from transformers import AutoTokenizer

# Example: Using a pre-trained tokenizer like BERT
src_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tgt_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

src_text = "can i have a large bbq pulled pork"
tgt_text = "(ORDER can i have (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING bbq pulled pork ) ) )"

# Tokenize the input and output
src_tokens = src_tokenizer(src_text, return_tensors="pt", padding=True, truncation=True)
tgt_tokens = tgt_tokenizer(tgt_text, return_tensors="pt", padding=True, truncation=True)

print(src_tokens)
print(tgt_tokens)

C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'input_ids': tensor([[  101,  2064,  1045,  2031,  1037,  2312, 22861,  4160,  2766, 15960,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  101,  1006,  2344,  2064,  1045,  2031,  1006, 10733,  8551,  2121,
          1006,  2193,  1037,  1007,  1006,  2946,  2312,  1007,  1006, 22286,
         22861,  4160,  2766, 15960,  1007,  1007,  1007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

class PizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data["train.SRC"][idx]
        tgt_text = self.data["train.TOP"][idx]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }


dataset = PizzaDataset(df, src_tokenizer, tgt_tokenizer)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [27]:
dataset.__getitem__(1)

{'src_input_ids': tensor([  101,  2312, 11345,  2007,  2665, 11565,  1998,  2007,  4469, 27233,
         26534,  3490,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'src_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]),
 'tgt_input_ids': tensor([  101,  1006,  2344,  1006, 10733,  8551,  2121,  1006,  2946,  2312,
          1007, 11345,  2007,  1006, 22286,  2665, 11565,  1007,  1998,  2007,
          1006,  3375,  1035, 22286,  1006, 11712,  4469,  1007,  1006, 22286,
         27233, 26534,  3490,  1007,  1007,  1007,  1007,   102,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [28]:
from transformers import BartForConditionalGeneration

# Initialize a seq2seq model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")


In [32]:
from tqdm import tqdm

# Initialize variables for tracking
running_loss = 0.0
total_batches = len(dataloader)
total_batches

76764

In [33]:

# Training Loop with Loss Tracking
for batch_idx, batch in enumerate(tqdm(dataloader, desc="Training Progress", unit="batch")):
    src_input_ids = batch["src_input_ids"]
    src_attention_mask = batch["src_attention_mask"]
    tgt_input_ids = batch["tgt_input_ids"]

    # Shift target for decoder input
    labels = tgt_input_ids.clone()
    labels[labels == tgt_tokenizer.pad_token_id] = -100  # Ignore padding for loss

    outputs = model(
        input_ids=src_input_ids,
        attention_mask=src_attention_mask,
        labels=labels
    )
    loss = outputs.loss
    running_loss += loss.item()
    
    # Print progress every 10 batches
    if (batch_idx + 1) % 10 == 0:
        avg_loss = running_loss / (batch_idx + 1)
        print(f"Batch {batch_idx + 1}/{total_batches}, Average Loss: {avg_loss:.4f}")

# Print final loss at the end
print(f"Final Average Loss: {running_loss / total_batches:.4f}")



Training Progress:   0%|          | 10/76764 [00:15<31:42:15,  1.49s/batch]

Batch 10/76764, Average Loss: 9.6387


Training Progress:   0%|          | 20/76764 [00:31<34:03:10,  1.60s/batch]

Batch 20/76764, Average Loss: 9.6066


Training Progress:   0%|          | 30/76764 [00:48<37:56:17,  1.78s/batch]

...

Batch 580/76764, Average Loss: 9.6234


Training Progress:   1%|          | 590/76764 [15:59<37:57:58,  1.79s/batch]

Batch 590/76764, Average Loss: 9.6231


Training Progress:   1%|          | 597/76764 [16:13<34:29:06,  1.63s/batch]


KeyboardInterrupt: 

In [34]:
input_text = "can i have a large bbq pulled pork"
input_tokens = src_tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Generate prediction
output_tokens = model.generate(input_ids=input_tokens["input_ids"], attention_mask=input_tokens["attention_mask"], max_length=50)
predicted_text = tgt_tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(f"Predicted: {predicted_text}")


Predicted: [unused1] can i have a large bbq pulled pork ship [unused410] a large mates large bb caucasian pulled pork [unused9] ship [unused410] initiatives large bb large bbhearted movement pork management jose pork ship § [unused410] [unused1]


In [ ]:
class DevPizzaDataset(Dataset):
    def __init__(self, data, src_tokenizer, tgt_tokenizer):
        self.data = data
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src_text = self.data[idx]["dev.SRC"]
        tgt_text = self.data[idx]["dev.TOP"]

        src_tokens = self.src_tokenizer(src_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)
        tgt_tokens = self.tgt_tokenizer(tgt_text, return_tensors="pt", padding="max_length", max_length=50, truncation=True)

        return {
            "src_input_ids": src_tokens["input_ids"].squeeze(0),
            "src_attention_mask": src_tokens["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_tokens["input_ids"].squeeze(0),
            "tgt_attention_mask": tgt_tokens["attention_mask"].squeeze(0),
        }
dev_dataset = DevPizzaDataset(dev, src_tokenizer, tgt_tokenizer)
dev_dataloader = DataLoader(dev_dataset, batch_size=1, shuffle=False)

In [ ]:
model.eval() 

predictions = []
ground_truths = []

with torch.no_grad():
    for batch in dev_dataloader:
        src_input_ids = batch["src_input_ids"]
        src_attention_mask = batch["src_attention_mask"]

        # Generate output sequence
        output_tokens = model.generate(
            input_ids=src_input_ids,
            attention_mask=src_attention_mask,
            max_length=100,  # Adjust as needed
            num_beams=4      # Use beam search for better predictions
        )

        # Decode predictions and ground truth
        predicted_text = tgt_tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        ground_truth_text = tgt_tokenizer.decode(batch["tgt_input_ids"][0], skip_special_tokens=True)

        predictions.append(predicted_text)
        ground_truths.append(ground_truth_text)

        print(f"Source: {src_tokenizer.decode(src_input_ids[0], skip_special_tokens=True)}")
        print(f"Prediction: {predicted_text}")
        print(f"Ground Truth: {ground_truth_text}\n")


In [ ]:
exact_matches = sum([1 for pred, gt in zip(predictions, ground_truths) if pred == gt])
accuracy = exact_matches / len(ground_truths)

print(f"Exact Match Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Tokenize predictions and ground truths
predictions_tokens = [pred.split() for pred in predictions]
ground_truths_tokens = [[gt.split()] for gt in ground_truths]  # Nested list for corpus_bleu

bleu_score = corpus_bleu(ground_truths_tokens, predictions_tokens)
print(f"BLEU Score: {bleu_score * 100:.2f}")


In [ ]:
for i, (pred, gt) in enumerate(zip(predictions, ground_truths)):
    if pred != gt:
        print(f"Mismatch {i + 1}:")
        print(f"Prediction: {pred}")
        print(f"Ground Truth: {gt}\n")
